In [5]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings

warnings.filterwarnings('ignore')

In [6]:
# parameters
customer_id = '5'
formatted_attribute = 'length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Curtains"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
# regex_pattern = r'(?i)(\b\d*\s?\-\s?Inch\s?x?\s?\d*\-?\s?\b)|(\s?\d*\s?\d+\/\d+\-\s?Inch\s?x\s?\d*\s?\d+\/\d+\s?\-\s?Inch\s?\b)|()'
regex_pattern = r'(?i)(\d+\-foot)|(\d+\s?foot)|()'


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [7]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [8]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

attribute      value    customers_name external_id  \
0        length    0.01 in  bedbathandbeyond    66272168   
1        length    0.01 in  bedbathandbeyond    67614349   
2        length    0.02 in  bedbathandbeyond    69210471   
3        length     0.1 in  bedbathandbeyond    69481838   
4        length     0.1 in  bedbathandbeyond    69481839   
...         ...        ...               ...         ...   
91643    length        n/a  bedbathandbeyond    77495078   
91644    length        n/a  bedbathandbeyond    77495082   
91645    length  Under 24"  bedbathandbeyond      206570   
91646    length  Under 24"  bedbathandbeyond     3284235   
91647    length  Under 24"  bedbathandbeyond     3288472   

                                                                                long_desc  \
0                                   SlipX Solutions® Floor to Ceiling PEVA Liner in Clear   
1                                          Lush Decor Animal Print Blanket Sets in Purple   
2                                                         Therapedic® Waterproof Underpad   
3                                                       Lush Decor Shower Curtain in Gray   
4                                                       Lush Decor Shower Curtain in Navy   
...                                                                                   ...   
91643                          BergHOFF® Leo 64 oz. Covered Glass Pasta Container in Pink   
91644  BergHOFF® Leo Tall Smart Seal Extra Large Food Storage Container with Lid in Green   
91645                                  Otello Honeycomb Pinch Pleat Window Curtain Panels   
91646                                                                Verso Window Valance   
91647                                              Waverly® Lovely Lattice Window Valance   

      resolution  
0       standard  
1       standard  
2       standard  
3       standard  
4       standard  
...          ...  
91643   standard  
91644   standard  
91645   standard  
91646   standard  
91647   standard  

[91648 rows x 6 columns]

In [9]:
# metric='in'
# range_string= """
# """

# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# list_values

# kwp=KeywordProcessor()
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts()

In [10]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [11]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute      value    customers_name external_id  \
0        length    0.01 in  bedbathandbeyond    66272168   
1        length    0.01 in  bedbathandbeyond    67614349   
2        length    0.02 in  bedbathandbeyond    69210471   
3        length     0.1 in  bedbathandbeyond    69481838   
4        length     0.1 in  bedbathandbeyond    69481839   
...         ...        ...               ...         ...   
91643    length        n/a  bedbathandbeyond    77495078   
91644    length        n/a  bedbathandbeyond    77495082   
91645    length  Under 24"  bedbathandbeyond      206570   
91646    length  Under 24"  bedbathandbeyond     3284235   
91647    length  Under 24"  bedbathandbeyond     3288472   

                                                                                long_desc  \
0                                   SlipX Solutions® Floor to Ceiling PEVA Liner in Clear   
1                                          Lush Decor Animal Print Blanket Sets in Purple   
2                                                         Therapedic® Waterproof Underpad   
3                                                       Lush Decor Shower Curtain in Gray   
4                                                       Lush Decor Shower Curtain in Navy   
...                                                                                   ...   
91643                          BergHOFF® Leo 64 oz. Covered Glass Pasta Container in Pink   
91644  BergHOFF® Leo Tall Smart Seal Extra Large Food Storage Container with Lid in Green   
91645                                  Otello Honeycomb Pinch Pleat Window Curtain Panels   
91646                                                                Verso Window Valance   
91647                                              Waverly® Lovely Lattice Window Valance   

      resolution matches  
0       standard      []  
1       standard      []  
2       standard      []  
3       standard      []  
4       standard      []  
...          ...     ...  
91643   standard      []  
91644   standard      []  
91645   standard      []  
91646   standard      []  
91647   standard      []  

[91648 rows x 7 columns]

In [12]:
df_foot=df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]
df_foot['matchez']=['10 ft', '15 ft', '3 ft', '4.5 ft', '6 ft', '7.5 ft', '7.5 ft', '9 ft', '8 ft']
df_foot

attribute value    customers_name external_id  \
80169    length   n/a  bedbathandbeyond    10664608   
80480    length   n/a  bedbathandbeyond    14897585   
84743    length   n/a  bedbathandbeyond    62240185   
88752    length   n/a  bedbathandbeyond    66862987   
88753    length   n/a  bedbathandbeyond    66862994   
88754    length   n/a  bedbathandbeyond    66863007   
88755    length   n/a  bedbathandbeyond    66863014   
88756    length   n/a  bedbathandbeyond    66863021   
91551    length   n/a  bedbathandbeyond    69654513   

                                                                           long_desc  \
80169  Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80480         Con-Tact® Grip 12-Inch x 15-Foot Ultra Shelf and Drawer Liner in Brown   
84743          Trinity 14-Inch x 3-Foot Non-Adhesive Frosted Shelf Liners (Set of 4)   
88752                          Elf Stor 4.5-Foot Christmas Tree Storage Cover in Red   
88753                            Elf Stor 6-Foot Christmas Tree Storage Cover in Red   
88754                          Elf Stor 7.5-Foot Christmas Tree Storage Cover in Red   
88755                              Elf Stor 7.5-Foot Christmas Tree Poly Storage Bag   
88756                                Elf Stor 9-Foot Christmas Tree Poly Storage Bag   
91551                                       simplehuman® 8-Foot Tension Shower Caddy   

      resolution    matches matchez  
80169   standard  [10-Foot]   10 ft  
80480   standard  [15-Foot]   15 ft  
84743   standard   [3-Foot]    3 ft  
88752   standard   [5-Foot]  4.5 ft  
88753   standard   [6-Foot]    6 ft  
88754   standard   [5-Foot]  7.5 ft  
88755   standard   [5-Foot]  7.5 ft  
88756   standard   [9-Foot]    9 ft  
91551   standard   [8-Foot]    8 ft

In [14]:
matches=df_foot
matches[curation_col] = matches['matchez']
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

attribute value    customers_name  external_id  \
80169    length   n/a  bedbathandbeyond  ="10664608"   
80480    length   n/a  bedbathandbeyond  ="14897585"   
84743    length   n/a  bedbathandbeyond  ="62240185"   
88752    length   n/a  bedbathandbeyond  ="66862987"   
88753    length   n/a  bedbathandbeyond  ="66862994"   
88754    length   n/a  bedbathandbeyond  ="66863007"   
88755    length   n/a  bedbathandbeyond  ="66863014"   
88756    length   n/a  bedbathandbeyond  ="66863021"   
91551    length   n/a  bedbathandbeyond  ="69654513"   

                                                                           long_desc  \
80169  Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80480         Con-Tact® Grip 12-Inch x 15-Foot Ultra Shelf and Drawer Liner in Brown   
84743          Trinity 14-Inch x 3-Foot Non-Adhesive Frosted Shelf Liners (Set of 4)   
88752                          Elf Stor 4.5-Foot Christmas Tree Storage Cover in Red   
88753                            Elf Stor 6-Foot Christmas Tree Storage Cover in Red   
88754                          Elf Stor 7.5-Foot Christmas Tree Storage Cover in Red   
88755                              Elf Stor 7.5-Foot Christmas Tree Poly Storage Bag   
88756                                Elf Stor 9-Foot Christmas Tree Poly Storage Bag   
91551                                       simplehuman® 8-Foot Tension Shower Caddy   

      resolution    matches matchez Q:length  
80169   standard  [10-Foot]   10 ft    10 ft  
80480   standard  [15-Foot]   15 ft    15 ft  
84743   standard   [3-Foot]    3 ft     3 ft  
88752   standard   [5-Foot]  4.5 ft   4.5 ft  
88753   standard   [6-Foot]    6 ft     6 ft  
88754   standard   [5-Foot]  7.5 ft   7.5 ft  
88755   standard   [5-Foot]  7.5 ft   7.5 ft  
88756   standard   [9-Foot]    9 ft     9 ft  
91551   standard   [8-Foot]    8 ft     8 ft

In [15]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [16]:
looks_good('Bed Bath & Beyond', attribute, df, matches)

In [34]:
df['matches'].explode().value_counts()

Foot    64
Name: matches, dtype: int64

In [10]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
    data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [11]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [12]:
def replace_none(data):
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
    data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [13]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")

In [14]:
def min_max(data):
    data['length']=data['length'].astype(int)
    data['width']=data['width'].astype(int)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [15]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split(' ')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [16]:
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                        matches  
80147   standard                     [40-Inch ]  
80169   standard                [24-Inch x 10-]  
80174   standard                [36-Inch x 72-]  
80188   standard  [ 12 1/2-Inch x 16 1/4-Inch ]  
80189   standard  [ 10 1/2-Inch x 12 1/2-Inch ]  
...          ...                            ...  
91620   standard                     [18-Inch ]  
91621   standard                      [7-Inch ]  
91622   standard                     [18-Inch ]  
91632   standard                     [60-Inch ]  
91642   standard                     [11-Inch ]  

[3225 rows x 7 columns]

In [17]:
split_col(df) #might throw an error and have to drop nane

In [18]:
df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                      matches        length         width  
80147   standard                     40-inch       40-inch              0  
80169   standard                24-inch x 10-       24-inch           10-  
80174   standard                36-inch x 72-       36-inch           72-  
80188   standard   12 1/2-inch x 16 1/4-inch    12 1/2-inch  16 1/4-inch   
80189   standard   10 1/2-inch x 12 1/2-inch    10 1/2-inch  12 1/2-inch   
...          ...                          ...           ...           ...  
91620   standard                     18-inch       18-inch              0  
91621   standard                      7-inch        7-inch              0  
91622   standard                     18-inch       18-inch              0  
91632   standard                     60-inch       60-inch              0  
91642   standard                     11-inch       11-inch              0  

[3225 rows x 9 columns]

In [19]:
df['length']=df['length'].astype(str)
df['width']=df['width'].astype(str)
df['length']=df['length'].apply(lambda quote: quote.replace('-', ''))
df['width']=df['width'].apply(lambda quote: quote.replace('-', ''))
df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                      matches   length    width  
80147   standard                     40-inch       40         0  
80169   standard                24-inch x 10-       24       10  
80174   standard                36-inch x 72-       36       72  
80188   standard   12 1/2-inch x 16 1/4-inch    12 1/2  16 1/4   
80189   standard   10 1/2-inch x 12 1/2-inch    10 1/2  12 1/2   
...          ...                          ...      ...      ...  
91620   standard                     18-inch       18         0  
91621   standard                      7-inch        7         0  
91622   standard                     18-inch       18         0  
91632   standard                     60-inch       60         0  
91642   standard                     11-inch       11         0  

[3225 rows x 9 columns]

In [20]:
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                      matches   length    width  
80147   standard                     40-inch       40         0  
80169   standard                24-inch x 10-       24       10  
80174   standard                36-inch x 72-       36       72  
80188   standard   12 1/2-inch x 16 1/4-inch    12 1/2  16 1/4   
80189   standard   10 1/2-inch x 12 1/2-inch    10 1/2  12 1/2   
...          ...                          ...      ...      ...  
91620   standard                     18-inch       18         0  
91621   standard                      7-inch        7         0  
91622   standard                     18-inch       18         0  
91632   standard                     60-inch       60         0  
91642   standard                     11-inch       11         0  

[3225 rows x 9 columns]

In [21]:
df[df['length'].astype(str)=='84 100']

attribute value    customers_name external_id  \
84124    length   n/a  bedbathandbeyond    47699618   

                                                                   long_desc  \
84124  Darcy 84-Inch 100% Blackout Grommet Top Window Curtain Panel in White   

      resolution      matches  length width  
84124    invalid  84-inch 100  84 100     0

In [22]:
float(sum(Fraction(s) for s in '12 1/2'.split()))

12.5

In [23]:
# a=['10 1/2', '12 1/2']
# # a = map(int, a)
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# c

In [24]:
error

NameError: name 'error' is not defined

In [25]:
a=df['length'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['length']=c
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                      matches  length    width  
80147   standard                     40-inch     40.0        0  
80169   standard                24-inch x 10-    24.0       10  
80174   standard                36-inch x 72-    36.0       72  
80188   standard   12 1/2-inch x 16 1/4-inch     12.5  16 1/4   
80189   standard   10 1/2-inch x 12 1/2-inch     10.5  12 1/2   
...          ...                          ...     ...      ...  
91620   standard                     18-inch     18.0        0  
91621   standard                      7-inch      7.0        0  
91622   standard                     18-inch     18.0        0  
91632   standard                     60-inch     60.0        0  
91642   standard                     11-inch     11.0        0  

[3225 rows x 9 columns]

In [26]:
a=df['width'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['width']=c
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                      matches  length  width  
80147   standard                     40-inch     40.0   0.00  
80169   standard                24-inch x 10-    24.0  10.00  
80174   standard                36-inch x 72-    36.0  72.00  
80188   standard   12 1/2-inch x 16 1/4-inch     12.5  16.25  
80189   standard   10 1/2-inch x 12 1/2-inch     10.5  12.50  
...          ...                          ...     ...    ...  
91620   standard                     18-inch     18.0   0.00  
91621   standard                      7-inch      7.0   0.00  
91622   standard                     18-inch     18.0   0.00  
91632   standard                     60-inch     60.0   0.00  
91642   standard                     11-inch     11.0   0.00  

[3225 rows x 9 columns]

In [ ]:
# x=df
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# x['new'] = c
# x

In [ ]:
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)

In [ ]:
# remove_fractions(df['length'])

In [ ]:
# x=df
# a=eval(x['length'][3:4].astype(str))
# # numbers = eval(a)

In [ ]:
# float(sum(Fraction(s) for s in a))

In [ ]:
# from fractions import Fraction
# float(sum(Fraction(s) for s in '  12 1/2'.split()))

In [ ]:
# x=df
# x['length'] = x['length'].apply(lambda quote: quote.replace('-', ''))
# x['length'] = x['length'].apply(lambda quote: quote.replace('inch', ''))
# # x['length'] = x['length'].apply(lambda x: x.str)
# x

In [ ]:
# x[['first', 'second']]=x['length'].str.split("/",expand=True,)
# x[['third', 'fourth']= x['first'].split()
# result = x['third'] + (float(x['first'])/float(x['second']))
# result

In [ ]:
# num,den = s.split( '/' )
# wh, num = num.split()
# result = wh + (float(num)/float(den))

In [ ]:
# a=x['length'][3:4]
# a.str.split(' ')
# b= [[" ".join(tag.split(" ")[:2]) , " ".join(tag.split(" ")[2:])] for tag in a][0]
# b

In [ ]:
# c=b[1]
# c.split('/')

In [ ]:
# b[0]
# b

In [ ]:
# a.apply(lambda x: sum(Fraction(x.split('/'))))

In [ ]:
# float(sum(Fraction(s) for s in x['length'].split()))
# x['length']=x['length'].apply(lambda a: float(sum(Fraction(a.split()))))

In [ ]:
# for i in x['length']:
#     if x['length'][i].find('/') > -1:
#         float(sum(Fraction(s) for s in x['length'][i].split()))
#     else:
#         x['length']

In [ ]:
# float(sum(Fraction(s) for s in x['length'][i].split()))
# .apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))

In [ ]:
# def fraction_combo(numbers):
#     if numbers.find('/') > -1:
#         numbers.apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))
#         #numbers = literal_eval(numbers)
#         #inte, fract = numbers.split(' ')
#         #nom, den = fract.split('/')
#         #tmp = float(nom)/float(den)
#         #num = int(inte) + tmp
#         return str(numbers)
#     else:
#         return numbers

In [ ]:
# a=x['length']

In [ ]:
# remove_fractions(df)

In [ ]:
# x['length'] = x['length'].apply(lambda x: remove_fractions(x))
# x.head

In [ ]:
# x['length'] = x['length'].apply(lambda x: x.split(' '))
# x.head()

In [ ]:
drop_characters(df)
min_max(df) #error base 10 when there is only a value for max and not min

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
# range function builder
string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
matches=df
matches

In [ ]:
matches[curation_col] = matches['Max'].apply(range_app)
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)